In [1]:
%autosave 0

Autosave disabled


In [3]:
import pymysql
import numpy as np
from pandas import DataFrame
import pandas as pd
import time
import smtplib
from email.mime.text import MIMEText
from email.header import Header
from email.mime.multipart import MIMEMultipart
from email.mime.application import MIMEApplication

In [5]:
# 连接数据库
db = pymysql.connect(host='dev.rrx.mysql01.srv',
                    db='jjd',
                    user='dev',
                    password='KRkFcVCbopZbS8R7',
                    charset='utf8')

# sql
sql = '''select ubi.full_name '出借人姓名',
ubi.telephone '出借人电话',
l.amount/100 '借条金额',
l.create_time '出借日期'
from loan l, user_basic_info ubi
where datediff(l.create_time, curdate()) = 0
and l.lender_uid = ubi.uid'''
cur = db.cursor()
cur.execute(sql)
data = cur.fetchall()
data1 = np.array(data)
if len(data1) == 0:
    data1 = [[0,0,0,0]]
else:
    data1
frame1 = DataFrame(data1,
                  columns=['出借人姓名','出借人电话','借条金额','出借日期'])

sql = '''select uid '用户ID',
area_code '地区编码',
login_cnt '登录次数'
from user_statis_ip
limit 20'''
cur.execute(sql)
data = cur.fetchall()
data2 = np.array(data)
if len(data2) == 0:
    data2 = [[0,0,0]]
else:
    data2
frame2 = DataFrame(data2,
                   columns=['用户ID','地区编码','登录次数'])

sql = '''select id '用户ID',
lend_cnt '出借次数',
lend_amount/100 '出借总额',
overdue_lend_cnt '被逾期借条数',
lend_30days_cnt '最后30天借条总数'
from user_statis_lender
limit 100'''
cur.execute(sql)
data = cur.fetchall()
data3 = np.array(data)
if len(data3) == 0:
    data3 = [[0,0,0,0,0]]
else:
    data3
frame3 = DataFrame(data3,
                  columns=['用户ID','出借次数','出借总额','被逾期借条数','最后30天借条总数'])
writer1 = pd.ExcelWriter('./jjd_loan_today.xlsx')
writer2 = pd.ExcelWriter('./jjd_lender.xlsx')

try:
    frame1.to_excel(writer1,
                   sheet_name='loan_today',
                   index=0)
    frame2.to_excel(writer2,
                   sheet_name='ip',
                   index=0)
    frame3.to_excel(writer2,
                   sheet_name='lender',
                   index=0)
    writer1.save()
    writer2.save()
    writer1.close()
    writer2.close()
except Exception as err:
    fillte = '导出失败' + str(err)
else:
    succeful = '导出成功'

In [8]:
sql = '''select date_format(create_time, '%Y-%m-%d') '日期',
count(id) '借条数',
count(distinct lender_uid) '出借人数',
sum(amount/100) '总金额'
from loan
where date_format(create_time, '%Y-%m-%d') >= '2019-07-01'
group by 1'''
cur.execute(sql)
data = cur.fetchall()
loan_cnt = np.array(data)
if len(loan_cnt) == 0:
    loan_cnt = [[0,0,0,0]]
else:
    loan_cnt
df_loan_cnt = DataFrame(loan_cnt,
                       columns=['日期','借条数','出借人数','总金额'])

In [10]:
writer3 = pd.ExcelWriter('./loan_cnt.xlsx')
df_loan_cnt.to_excel(writer3,
                    sheet_name='loan_cnt',
                    index=0)

In [11]:
writer3.save()
writer3.close()

In [13]:
df = pd.read_excel('./loan_cnt.xlsx')
df

,日期,借条数,出借人数,总金额
0,2019-07-01,9,3,2037.00
1,2019-07-02,9,3,9898.00
2,2019-07-03,4,2,6604.95
3,2019-07-05,6,2,9908.00
4,2019-07-08,3,1,7709.00
5,2019-07-09,8,1,5200.00
6,2019-07-10,3,1,2202.43
7,2019-07-15,3,1,1200.00
8,2019-07-17,4,3,900.00
9,2019-07-18,13,6,3995.00


In [24]:
df_to_html = df.to_html(index=False)

In [25]:
df_to_html

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th>日期</th>\n      <th>借条数</th>\n      <th>出借人数</th>\n      <th>总金额</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <td>2019-07-01</td>\n      <td>9</td>\n      <td>3</td>\n      <td>2037.00</td>\n    </tr>\n    <tr>\n      <td>2019-07-02</td>\n      <td>9</td>\n      <td>3</td>\n      <td>9898.00</td>\n    </tr>\n    <tr>\n      <td>2019-07-03</td>\n      <td>4</td>\n      <td>2</td>\n      <td>6604.95</td>\n    </tr>\n    <tr>\n      <td>2019-07-05</td>\n      <td>6</td>\n      <td>2</td>\n      <td>9908.00</td>\n    </tr>\n    <tr>\n      <td>2019-07-08</td>\n      <td>3</td>\n      <td>1</td>\n      <td>7709.00</td>\n    </tr>\n    <tr>\n      <td>2019-07-09</td>\n      <td>8</td>\n      <td>1</td>\n      <td>5200.00</td>\n    </tr>\n    <tr>\n      <td>2019-07-10</td>\n      <td>3</td>\n      <td>1</td>\n      <td>2202.43</td>\n    </tr>\n    <tr>\n      <td>2019-07-15</td>\n      

In [18]:
type(df_to_html)

str

In [26]:
content = '''<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>今借到借条统计</title>
    <meta name="viewport" content="width=device-width,initial-scale=1,maximum-scale=1,user-scalable=no">
    <style type="text/css">
        table {
            border-collapse:collapse;
        }
        table, td, th {
            border: 1px solid black;
            text-align: center;
        }
    </style>
</head>
<body>
<p>Hi, all</p>''' + df_to_html + '''
</body>
</html>'''



In [27]:
# 发送邮箱服务器
smtpserver = 'smtp.exmail.qq.com'

# 发送邮箱用户名和密码
user = "qufang@jinjiedao.cn"
password = "Qf080501"

# 发送和接收邮箱
sender = "qufang@jinjiedao.cn"
receive = ["qufang@jinjiedao.cn"]

ctime = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())
message = MIMEMultipart()
message['From'] = Header("qufang", 'utf-8')
message['To'] = ','.join(receive)
subject = "今借到今日借条信息" + ctime + ""
message['Subject'] = Header(subject, 'utf-8')

# 邮件正文内容
# message.attach(MIMEText('Hi，附件为D1风控新增用户数据，请查收，工作愉快~',
#                        'plain',
#                        'utf-8'))
message.attach(MIMEText(content, 'html', 'utf-8'))

part1 = MIMEApplication(open('./jjd_loan_today.xlsx','rb').read())
part1.add_header('Content-Disposition',
                'attachment',
                filename='jjd_loan_today.xlsx')
message.attach(part1)

part2 = MIMEApplication(open('./jjd_lender.xlsx','rb').read())
part2.add_header('Content-Disposition',
                'attachment',
                filename='jjd_lender.xlsx')
message.attach(part2)

# SSL协议端口号要使用465
smtp = smtplib.SMTP_SSL(smtpserver, 465)

# HELO向服务器标志用户身份
smtp.helo(smtpserver)

# 服务器返回结果确认
smtp.ehlo(smtpserver)

# 登录邮箱服务器用户名密码
smtp.login(user, password)

print("Send email start...")
smtp.sendmail(sender,
             receive,
             message.as_string())
smtp.quit()
print("email send end !")

Send email start...
email send end !


In [ ]:
# 封装成函数

def getdata(sql):
    conn = pymysql.connect(host='dev.rrx.mysql01.srv',
                           db='jjd',
                           user='dev',
                           password='KRkFcVCbopZbS8R7',
                           charset='utf8')
    cur = conn.cursor()
    try:
        cur.execute(sql)
        result = cur.fetchall()
        return str(result)
    except Exception as e:
        print(e)
    finally:
        conn.commit()
        cur.close()
        conn.close()
        
sql = '''select ubi.full_name '出借人姓名',
ubi.telephone '出借人电话',
l.amount/100 '借条金额',
l.create_time '出借日期'
from loan l, user_basic_info ubi
where datediff(l.create_time, curdate()) = 0
and l.lender_uid = ubi.uid'''
data1 = getdata(sql)